





# Investing in the King County Housing Market 
## Foundations of a clever investment decision

The data contains the King County House prices from May 2014 - to May 2015. It contains the selling prices, information on 18 house features and the date of sale. 
Our stakeholder is interested in buying und selling houses for the highest possible profit. Therefore things as social responsibility or a sustainable urban development are counterproductive. Her mantra: 'If I don't do it, someone else will.'.
To match her interest in the housing market we need to provide data where to find cheap objects with high profit margins.

## Structure
1. Preparing the data
2. Understanding the data
3. Insights 
4. Recommendations for our Stakeholder


Hypotheses:
1. The nearer a district is located to the city center the higher housing prices get.
2. House prices increased in the years 2014 - 2015.
3. When there is more space for living the prices increase.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt
import datetime 
import geojson



In [ ]:
# Import data
king_county = pd.read_csv("data/King_County_House_prices_dataset.csv")



# Preparing the data
First step: Cleaning the data. We check for duplicates, missing values and correct the datatypes.


In [ ]:
# all columns and datatype
king_county.info()

In [ ]:
#check dataframe shape
king_county.shape

We can see that there are missing values in column 'waterfront','view' and 'yr_renovated'. We can also assume that 'sqft_basement' should be a float. We should change the 'yr_built' and 'yr_renovated' into datetime objects. We should also check if there are duplicates, e.g. in the 'id' column. 

In [ ]:
#check wether there are duplicates for 'id'
king_county['id'].nunique 

There are no duplicates of 'id' in the data set. Next step check for missing values.

In [ ]:
#check missing values
king_county.isnull().sum()

So there are 2376 missing values in 'waterfront', 63 in 'view' and 3842 in 'yr_renovated'. 

### waterfront: missing values

In [ ]:
#insight in missing values of 'waterfront'
print(king_county['waterfront'].nunique())
king_county['waterfront'].value_counts()

Since the majority of the values of 'waterfront' is 0 (absence of waterfront), we can replace the missing values wiht 0. 

In [ ]:
king_county['waterfront'] = king_county['waterfront'].fillna(0)
#check
king_county['waterfront'].isnull().sum()

### view: missing values

In [ ]:
print(king_county['view'].nunique())
king_county['view'].value_counts()

In [ ]:
#we replace missing values with 0 as it is the majority.
king_county['view'] = king_county['view'].fillna(0)
#check
king_county['view'].isnull().sum()

### yr_renovated: missing values

In [ ]:
#check
print(king_county['yr_renovated'].nunique())
king_county['yr_renovated'].value_counts()

In [ ]:
#again we can replace missing values with 0 as it looks like the majority of the values ist 0.
king_county['yr_renovated'] = king_county['yr_renovated'].fillna(0)
#check
king_county['yr_renovated'].isnull().sum()

### correction of datatypes: 'sqft_basement' & 'date'

In [ ]:
#check out the sqft_basement values to understand, why it isn't a float
king_county['sqft_basement'].sort_values(ascending=True)

In [ ]:
#how many '?' are there? 
len(king_county[king_county['sqft_basement'] == '?'])

In [ ]:
#as we can see there are 454 '?' instead of integers. We can calculate new values for sqft_basement.
for i in list(king_county[king_county['sqft_basement'] == '?'].index.values):
    king_county.loc[i, 'sqft_basement'] = king_county.loc[i, 'sqft_living'] - king_county.loc[i, 'sqft_above']
#check
len(king_county[king_county['sqft_basement'] == '?'])

In [ ]:
#correct to float
king_county['sqft_basement'] = king_county['sqft_basement'].astype('float64')
#check
king_county['sqft_basement'].dtypes

date

In [ ]:
#is date datetime object?
isinstance(king_county['date'][0], datetime.date)

In [ ]:
#convert to datetime object
king_county['date'] = pd.to_datetime(king_county['date'])
#check
isinstance(king_county['date'][0], datetime.date)


In [ ]:

# Create new column for year
king_county['year'] = king_county.date.dt.year

# Create new column for day-year-month
king_county['day_year_month'] = king_county['date'].dt.strftime('%d/%m/%Y')

king_county.head(2)

## Understanding the data
In the next step we get to know the data a little further. Therefore we look at the descriptive statistics of the dataset and some initial visualizations to get a better understanding of the dataset.

In [ ]:
#getting to know the data
king_county.head(5)

In [ ]:
#getting to know the data
king_county.describe()

In [ ]:
#linearity scan
sns.pairplot(king_county, diag_kind = 'hist', corner = True)

When we scroll in we can see linearity:
- 'sqft_living' vs. 'price', 'sqft_living' vs. 'bathrooms', 'sqft_living' vs. 'sqft_above', 'sqft_living' vs. 'sqft_living15'
'sqft_above' vs. 'grade'
- 'sqft_living15' vs. 'grade', 'sqft_living15' vs. 'sqft_above'
- 'price' vs. 'bedrooms', 'price' vs. 'bathrooms', 'price' vs. 'grade', 'price' vs. 'sqft_above', 'price' vs. 'sqft_living15'

In [ ]:
#split data in 2 different groups, categorial and continuous, to provide a more precise overview
continuous = ['price', 'sqft_living','sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
categoricals = ['price', 'bedrooms', 'bathrooms', 'floors', 'waterfront', 'condition', 'grade']

In [ ]:
#continuous data
sns.pairplot(king_county[continuous], diag_kind = 'hist', corner=True)

In [ ]:
#categorial data
sns.pairplot(king_county[categoricals], diag_kind= 'hist', corner = 'True')


Relationships regarding the price: 
Categorial variables: As we can see 'bedrooms', 'grade', 'bathrooms' are in a linear relationship with 'price'. Of the continuous variables there is a clear linear relationship with 'sqft_living'.

In [ ]:
#closer look at the distribution of the price variable
import plotly.figure_factory as ff
fig = px.histogram(king_county, x="price", marginal="box", title='price distribution')
fig.show()


We have a minimum price of 78.000$ , a maximum price of 7.7 Million$. The 0.25 quantil is at about 322.00$, the median house price is at about 450.000$. Most of the observations are between 320.000 and 340.000$. 

## Insights
As our stakeholder is interested in high profit margins further examination is provided on the dependencies of the housing price. We are especially interested in the location of cheap houses and want to examine the different districts of King County.

In [ ]:
#zipcode with lowest price
#group df by the median of the prices and sort it
lowest_prices = king_county.groupby('zipcode').median().sort_values('price').reset_index()
lowest_prices

In [ ]:
#now here is the geographical plot
#imported gj above
#load zipcodes from the area from .geojson file (get from link)
with open ('http://data-seattlecitygis.opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson') as f:
    gj = geojson.load(f)
#zipcodes are added as ID for each zipcode area in the gj array. Plotly can now automatically link the zip code data to theappropiate area.

for i in range(len(gj['features'])):
    gj['features'][i]['id'] = gj['features'][i]['properties']['ZIPCODE']


In [ ]:
#plot
fig = px.choropleth_mapbox(lowest_prices, geojson=gj, locations='zipcode', color='price', title='Median houseprices / district',
                            color_continuous_scale='inferno_r',
                            range_color=(230000, 2000000),
                            mapbox_style = 'open-street-map',
                            zoom=8.1,
                            center={'lat' : 47.460, 'lon' : -122.014},
                            opacity=0.5,
                            hover_data=['price', 'zipcode'],
                            labels = {'price: median house prices'},
                            width=750,
                            height=500
                            )
fig.show()

##

We can see that the districts on the easternside of the Lake Washington are the districts with the highest prices. The 'Mercer Island', 'Medina' and 'Clyde Hill' belong to the areas with the highest house prices. After examining Google Maps we can state that the city center with possibilities to shop, eat out and enjoy life is located on the western side of the Lake Washington. 
We should take a closer look at the cheapest districts.

In [ ]:
#zipcode change into string to get a catergorial data to plot
king_county['zipcode'] = king_county['zipcode'].map(str)

In [ ]:
#data grouped by zipcode 
king_county.groupby('zipcode').count()

In [ ]:
#zipcode with lowest price
lowest_prices = king_county.groupby('zipcode').median().sort_values('price').reset_index()
lowest_prices

In [ ]:
#decide that price <300.000$ as the cheapest 25% of houses is suitable for our stakeholder. 
# we create a list with all the zipcodes with a median price < 300.000$ 
list_lowest_price = lowest_prices.query('price <= 300000').zipcode.tolist()
list_lowest_price


In [ ]:
#the zipcodes of the original dataframe are filtered with the list 
#dataframe with the zipcodes of the districts with median prices < 300.000 $:
df_cheapest_zipcodes = king_county[king_county['zipcode'].isin(list_lowest_price)]
df_cheapest_zipcodes

In [ ]:
#there are 15 zipcodes with median house prices < 300.000$
#data frame for the 15 cheapest zipcodes incl median price: 
df_cheapest_zipcode_median_price = df_cheapest_zipcodes.groupby('zipcode').median().sort_values('price').reset_index()
df_cheapest_zipcode_median_price

In [ ]:
#plot the 15 cheapest districts
fig = px.choropleth_mapbox(df_cheapest_zipcode_median_price, geojson=gj, locations='zipcode', color='price', title='districts with median prices < 300.000$',
                            color_continuous_scale='inferno',
                            range_color=(230000, 300000),
                            mapbox_style = 'open-street-map',
                            zoom=8.1,
                            center={'lat' : 47.460, 'lon' : -122.014},
                            opacity=0.5,
                            hover_data=['price', 'zipcode'],
                            labels = {'price: median house prices'},
                            width=750,
                            height=500
                            )

fig.show()

## Development of the prices in the cheapest districts 
Because our steakholder wants buy and resell with maximal profit it is useful to look at the price development in the cheapest districts.

In [ ]:
#development of median price in all cheap districts
df_cheapest_zipcodes.groupby('year').price.median()


In all cheap districts the median price increased from year 2014 to year 2015.

In [ ]:
#iterating through original dataframe and creating two new ones: one for 2014 and one for 2015
king_county_2014 = king_county.query('year == 2014')
king_county_2014.head(2)



In [ ]:
king_county_2015 = king_county.query('year == 2015')
king_county_2015.head(2) 

In [ ]:
#changing the datatype of zipcode to be able to merge the dataframes
king_county['zipcode']= king_county['zipcode'].astype('string')
king_county.dtypes

In [ ]:
#iterating through those new dataframes and creating new ones with cheapest zipcodes
kc_2014_cheap = king_county_2014[king_county_2014['zipcode'].isin(list_lowest_price)]
kc_2014_cheap

In [ ]:
kc_2015_cheap = king_county_2015[king_county_2015['zipcode'].isin(list_lowest_price)]
kc_2015_cheap

For the year 2014 we have 2663 prices and for the year 2015 we have 1382 prices. Although there are just half the observations in 2015 we can compare the median price per year and district to get an overview of the price development.

In [ ]:
median_cheaps_2014 = kc_2014_cheap.groupby('zipcode').mean().sort_values('price').round()
median_cheaps_2014.head()

In [ ]:
median_cheaps_2015 = kc_2015_cheap.groupby('zipcode').mean().sort_values('price').round(0)
median_cheaps_2015.head()


In [ ]:
#join the two dataframes on ID (which is the zipcode) with the median prcies of 2014 and 2015 to plot them.
cheap_joined = median_cheaps_2014.join(median_cheaps_2015, how ="inner", lsuffix= 'left', rsuffix= 'right')
cheap_joined

In [ ]:
#plot the prices of 2014/2015 per zipcode
fig = px.bar(cheap_joined, y= ['priceleft', 'priceright'], barmode = 'group', title= 'Price Development 2014 - 2015')


for idx in range(len(fig.data)):
    fig.data[idx].x = ['Tukwila', 'Auburn', 'Kent', 'Lakeland', 'Seatac/Tukwila', 'Burien', 'Federal Way', 'Federal Way/ East', 'Kent/Southeast', 'Kent/ East Hill', 'Renton', 'Brin Mawr-Skyway', 'Covington', 'Des Moines', 'Center']

fig.update_layout(
    font_family="Arial",
    title_font_size= 30,
    font_color="grey",
    title_font_family="Arial",
    title_font_color="grey",
    legend_title_font_color="grey"
)
fig.update_xaxes(title_font_family="Arial", title= 'District')
fig.update_yaxes(title_font_family="Arial", title= 'Median Price')

newnames = {'priceleft':'median price 2014', 'priceright': 'median price 2015'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

fig.show()

In [ ]:
#calculate the percentage of the price increase
def create_percentage(priceleft):
        for i in cheap_joined['priceleft']:

                price_percentage = ((cheap_joined['priceright'] - cheap_joined['priceleft']) / cheap_joined['priceleft']) * 100
                return price_percentage

create_percentage('priceleft')
        


In [ ]:
#add the price_percentage to the dataframe
cheap_joined['price_percentage']= (create_percentage('priceleft'))
cheap_joined.head(2)

In [ ]:
#plot price percentage increase from 2014 to 2015
fig = px.bar(cheap_joined, y= ['price_percentage'], title= 'price increase (percentage)')


for idx in range(len(fig.data)):
    fig.data[idx].x = ['Tukwila', 'Auburn', 'Kent', 'Lakeland', 'Seatac/Tukwila', 'Burien', 'Federal Way', 'Federal Way/ East', 'Kent/Southeast', 'Kent/ East Hill', 'Renton', 'Brin Mawr-Skyway', 'Covington', 'Des Moines', 'Center']

fig.update_layout(
    font_family="Arial",
    title_font_size= 30,
    font_color="grey",
    title_font_family="Arial",
    title_font_color="grey",
    legend_title_font_color="grey"
)
fig.update_xaxes(title_font_family="Arial", title= 'District')
fig.update_yaxes(title_font_family="Arial", title= 'price percentage')

#newnames = {'priceleft':'median price 2014', 'priceright': 'median price 2015'}
#fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

fig.show()

Observation:
In Tukwila we can observe an increase of housing prices of 9.5% in one year. 
In Seatac/ Tukwila it is 8.4%.
Overall there are only two districts with significant decreasing prcies: Burien and Des Moines.
